In [1]:
import random
import pandas as pd
import numpy as np
import time

In [2]:
def diff(Rule1, Rule2):
    if Rule1[-1] != Rule2[-1]:
        return -1
    m = len(Rule1)
    count = 0
    for i in range(m-1):
        if Rule1[i] == Rule2[i]:
            count += 1
    return count/m

In [3]:
def sampling(base, ran, e, k = 2):
    

    num = len(base)
    R = []
    list_index = []
    while (len(R)<num*ran/100):
        index = random.randrange(num)
        while (index in list_index):
            index = random.randrange(num)
        # list_index.append(index)
        T = []
        T.append(index)
        for i in range(index-k,index+k+1):
            if i < num:
                if i in list_index:
                    continue
                else:
                    if diff(base[i],base[index]) < 1 - e:
                        T.append(i)
        for i in T:
            temp = 0
            for j in range(len(R)):
                if diff(base[i],R[j]) < 1 - e:
                    continue
                else:
                    temp = 1
            if temp:
                T.remove(i)
        for i in T:
            R.append(base[i])
            list_index.append(i)
    
    return R



In [4]:
def combination(k, n):
    if k == 0 or k == n:
        return 1
    if k == 1:
        return n
    return combination(k - 1, n - 1) + combination(k, n - 1)


def caculateA(base):
    colum = len(base[0])
    row = len(base)
    A = np.zeros((row, combination(4, colum - 1)))

    for r1 in range(row):
        k = [0] * combination(4, colum - 1)
        temp = 0
        for a in range(0, colum - 4):
            for b in range(a + 1, colum - 3):
                for c in range(b + 1, colum - 2):
                    for d in range(c + 1, colum - 1):
                        for r2 in range(row):
                            if base[r1][a] == base[r2][a] and base[r1][b] == base[r2][b] and base[r1][c] == base[r2][c] and base[r1][d] == base[r2][d]:
                                k[temp] += 1

                        A[r1][temp] = k[temp] / row
                        temp += 1
    print("done A")
    return A


def caculateM(base):
    colum = len(base[0])
    row = len(base)
    M = np.zeros((row, colum - 1))
    for t1 in range(row):
        k = [0] * (colum - 1)
        temp = 0
        for i in range(colum - 1):
            for t2 in range(row):
                if base[t1][i] == base[t2][i] and base[t1][colum - 1] == base[t2][colum - 1]:
                    k[temp] += 1
            M[t1][temp] = k[temp] / row
            temp += 1

    return M


def caculateB(base, A, M):
    colum = len(base[0])
    row = len(base)
    B = np.zeros((row, combination(3, colum - 1)))

    for r in range(row):
        temp = 0
        for a in range(0, colum - 3):
            for b in range(a + 1, colum - 2):
                for c in range(b + 1, colum - 1):
                    B[r][temp] = sum(A[r]) * min(M[r][a], M[r][b], M[r][c])
                    temp += 1
    print("done B")
    return B


def caculateC(base, B):
    colum = len(base[0])
    row = len(base)
    cols = 2 * combination(3, colum - 1)
    C = np.zeros((row, cols))

    for r1 in range(row):
        temp = 0
        for i in range(2):
            for a in range(0, (colum - 3)):
                for b in range(a + 1, (colum - 2)):
                    for c in range(b + 1, (colum - 1)):
                        for r2 in range(row):
                            if base[r1][a] == base[r2][a] and base[r1][b] == base[r2][b] and base[r1][c] == base[r2][c] and base[r2][colum - 1] == i:
                                C[r1][temp] += B[r2][temp % combination(3, colum - 1)]
                        # print(temp,":",temp//combination(3,colum-1))
                        temp += 1
    print("done C")
    return C

In [5]:
def FISA(base, C, list):
    colum = len(base[0])
    row = len(base)

    cols = combination(3, (colum - 1))
    C0 = [0] * cols
    C1 = [0] * cols

    t = 0
    for a in range(0, colum - 3):
        for b in range(a + 1, colum - 2):
            for c in range(b + 1, colum - 1):
                for r in range(row-1):
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 0:
                        C0[t] = C[r][t + 0 * cols]
                        # break
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 1:
                        C1[t] = C[r][t + 1 * cols]
                        # break
                t += 1
    #print(t)

    D0 = max(C0) + min(C0)
    D1 = max(C1) + min(C1)

    #print(D0, max(C0), min(C0))
    #print(D1, max(C1), min(C1))
    #print(D2, max(C2), min(C2))
    if D0 > 3*D1:
        return 0, D0/(D0+D1)
    else:
        return 1, D1/(D0+D1)

In [6]:
def Acc(A,B):
    result = 0

    for i in range(len(A)):
        if int(A[i]) - int(B[i]) == 0:
            result += 1

    return round(result*100/len(A), 2)


def Tprecision(Pre,Act):
    result = 0
    TP = 0
    FP = 0

    for i in range(len(Pre)):
        if int(Pre[i]) == 1 and int(Act[i]) == 1:
            TP +=1
        if int(Pre[i]) == 1 and int(Act[i]) == 0:
            FP +=1

    #return str(TP) +  " : " + str(TP+FP)

    if TP:
        return round(100*TP/(TP+FP),2)
    else:
        if FP:
            return 0
        else:
            return None


def Trecall(Pre,Act):
    result = 0
    TP = 0
    FN = 0

    for i in range(len(Pre)):
        if int(Pre[i]) == 1 and int(Act[i]) == 1:
            TP +=1
        if int(Pre[i]) == 0 and int(Act[i]) == 1:
            FN +=1

    #return str(TP) +  " : " + str(TP+FN)

    if TP:
        return round(100*TP/(TP+FN),2)
    else:
        if FN:
            return 0
        else:
            return None

# listAcc = []
# listPre = []
# listRe = []
# timeTest = []
# timeUpdate = []
# listRank = []
# res = []

def testAccuracy(base,Te,C):
    test = Te
    X = np.zeros(len(test))
    ddd = np.zeros(len(test))
    X_test = np.array(test).T[-1]
    for i in range(len(test)):
        X[i], ddd[i] = FISA(base, C, test[i])
        # listRank.append(ddd[i])
        #print(test[i])
    # for dd in ddd:
    #     print(dd)
    # listRank.append(ddd)

    #print(X)
    #print(X_test)
    # res.append(X)
    # listAcc.append(Acc(X,X_test))
    # listPre.append(Tprecision(X,X_test))
    # listRe.append(Trecall(X,X_test))

    return Acc(X,X_test)

In [7]:
def scenario_FKGS(ran, e):

    traindf = pd.read_csv('train.csv')
    testdf = pd.read_csv('test.csv')
    train = traindf.values.tolist()
    test = testdf.values.tolist()


    sampling_time = []
    train_time = []
    test_time = []
    accuracy = []
    for i in range(10):
        start = time.time()
        base = sampling(ran=ran, base=train, e=e)
        totalTime = time.time() - start
        sampling_time.append(totalTime)
        print(f'len base {i}:', len(base))


        start = time.time()
        A = caculateA(base)
        M = caculateM(base)
        B = caculateB(base,A,M)
        C = caculateC(base,B)
        totalTime = time.time() - start
        train_time.append(totalTime)


        start = time.time()
        acc = testAccuracy(base,test,C)
        totalTime = time.time() - start
        accuracy.append(acc)
        test_time.append(totalTime)


    print('sampling_time:', sampling_time)
    print('train_time:', train_time)
    print('test_time:', test_time)
    print('accuracy:', accuracy)


In [8]:
def scenario_random(ran):

    traindf = pd.read_csv('train.csv')
    testdf = pd.read_csv('test.csv')
    test = testdf.values.tolist()

    sampling_time = []
    train_time = []
    test_time = []
    accuracy = []
    for i in range(10):
        
        start = time.time()
        train = traindf.sample(n=int(len(traindf)*ran/100))
        base = train.values.tolist()
        totalTime = time.time() - start
        sampling_time.append(totalTime)
        print(f'len base {i}:', len(base))


        start = time.time()
        A = caculateA(base)
        M = caculateM(base)
        B = caculateB(base,A,M)
        C = caculateC(base,B)
        totalTime = time.time() - start
        train_time.append(totalTime)


        start = time.time()
        acc = testAccuracy(base,test,C)
        totalTime = time.time() - start
        accuracy.append(acc)
        test_time.append(totalTime)

    print('sampling_time:', sampling_time)
    print('train_time:', train_time)
    print('test_time:', test_time)
    print('accuracy:', accuracy)


In [9]:
scenario_FKGS(10, 0.1)

len base 0: 1106
done A
done B
done C
len base 1: 1109
done A
done B
done C
len base 2: 1108
done A
done B
done C
len base 3: 1107
done A
done B
done C
len base 4: 1105
done A
done B
done C
len base 5: 1109
done A
done B
done C
len base 6: 1108
done A
done B
done C
len base 7: 1107
done A
done B
done C
len base 8: 1108
done A
done B
done C
len base 9: 1109
done A
done B
done C
sampling_time: [0.2600069046020508, 0.23199939727783203, 0.24885845184326172, 0.2349998950958252, 0.23938846588134766, 0.23899126052856445, 0.24200081825256348, 0.23699617385864258, 0.2414722442626953, 0.23921418190002441]
train_time: [31.506623029708862, 31.739449739456177, 31.14306616783142, 31.15798783302307, 33.204843282699585, 33.19451022148132, 33.51661396026611, 32.38435888290405, 32.87842297554016, 33.5699782371521]
test_time: [33.19203519821167, 33.09059524536133, 33.1375207901001, 33.81823801994324, 33.7563271522522, 33.898218393325806, 33.83402705192566, 33.80523180961609, 33.71794867515564, 34.1023049

In [10]:
scenario_FKGS(10, 0.2)

len base 0: 1105
done A
done B
done C
len base 1: 1105
done A
done B
done C
len base 2: 1105
done A
done B
done C
len base 3: 1106
done A
done B
done C
len base 4: 1106
done A
done B
done C
len base 5: 1105
done A
done B
done C
len base 6: 1108
done A
done B
done C
len base 7: 1108
done A
done B
done C
len base 8: 1106
done A
done B
done C
len base 9: 1106
done A
done B
done C
sampling_time: [0.263608455657959, 0.25600552558898926, 0.25777196884155273, 0.25625181198120117, 0.2530069351196289, 0.2581465244293213, 0.26040196418762207, 0.2637789249420166, 0.2540006637573242, 0.2560079097747803]
train_time: [26.98490285873413, 26.800354719161987, 27.433286905288696, 27.643937826156616, 27.244518041610718, 27.570192337036133, 27.295648336410522, 27.432748794555664, 27.34345507621765, 27.02359890937805]
test_time: [32.604233503341675, 32.67816710472107, 32.94248723983765, 33.10547876358032, 32.664320945739746, 32.82617115974426, 32.76816129684448, 32.995471239089966, 32.87534546852112, 32.80

In [11]:
scenario_FKGS(10, 0.3)

len base 0: 1105
done A
done B
done C
len base 1: 1105
done A
done B
done C
len base 2: 1107
done A
done B
done C
len base 3: 1107
done A
done B
done C
len base 4: 1105
done A
done B
done C
len base 5: 1105
done A
done B
done C
len base 6: 1105
done A
done B
done C
len base 7: 1105
done A
done B
done C
len base 8: 1106
done A
done B
done C
len base 9: 1106
done A
done B
done C
sampling_time: [0.29199790954589844, 0.28900909423828125, 0.276993989944458, 0.2929995059967041, 0.2799999713897705, 0.2779998779296875, 0.2916290760040283, 0.27900052070617676, 0.2870051860809326, 0.2803640365600586]
train_time: [27.1747465133667, 26.054755926132202, 26.56199860572815, 26.613518953323364, 26.886077880859375, 26.18720769882202, 26.25785541534424, 27.0478355884552, 27.248600482940674, 26.800175666809082]
test_time: [32.5234956741333, 31.97943377494812, 32.19903922080994, 32.06234312057495, 32.01394462585449, 31.978906393051147, 32.092358350753784, 32.19153356552124, 32.083887338638306, 31.95894765

In [12]:
scenario_FKGS(15, 0.1)

len base 0: 1659
done A
done B
done C
len base 1: 1661
done A
done B
done C
len base 2: 1662
done A
done B
done C
len base 3: 1661
done A
done B
done C
len base 4: 1661
done A
done B
done C
len base 5: 1660
done A
done B
done C
len base 6: 1660
done A
done B
done C
len base 7: 1661
done A
done B
done C
len base 8: 1657
done A
done B
done C
len base 9: 1662
done A
done B
done C
sampling_time: [0.5364391803741455, 0.5385091304779053, 0.5308091640472412, 0.5290014743804932, 0.5296978950500488, 0.527998685836792, 0.5309317111968994, 0.5219974517822266, 0.5385510921478271, 0.5503835678100586]
train_time: [71.62685680389404, 72.89349246025085, 70.28131985664368, 70.92470788955688, 70.51504874229431, 71.79411053657532, 71.96339893341064, 70.14768171310425, 68.5266706943512, 70.31425070762634]
test_time: [49.79776167869568, 49.9121949672699, 49.89693021774292, 49.63499665260315, 49.773930072784424, 49.925942182540894, 49.739259481430054, 49.70067858695984, 47.99212026596069, 49.795884132385254

In [13]:
scenario_FKGS(15, 0.2)

len base 0: 1658
done A
done B
done C
len base 1: 1657
done A
done B
done C
len base 2: 1657
done A
done B
done C
len base 3: 1658
done A
done B
done C
len base 4: 1660
done A
done B
done C
len base 5: 1659
done A
done B
done C
len base 6: 1659
done A
done B
done C
len base 7: 1659
done A
done B
done C
len base 8: 1658
done A
done B
done C
len base 9: 1659
done A
done B
done C
sampling_time: [0.5800611972808838, 0.5532033443450928, 0.5647141933441162, 0.5821542739868164, 0.5597434043884277, 0.5719988346099854, 0.5728452205657959, 0.5820753574371338, 0.5895586013793945, 0.5781822204589844]
train_time: [58.52577304840088, 59.04879283905029, 57.25052833557129, 59.09395480155945, 59.2384352684021, 60.2806191444397, 59.85166549682617, 59.04303956031799, 60.76476168632507, 58.03592538833618]
test_time: [47.54804039001465, 47.70123553276062, 46.2868971824646, 47.56790590286255, 47.9577362537384, 48.28744029998779, 48.024375677108765, 47.94218397140503, 47.96346163749695, 47.571577072143555]
a

In [14]:
scenario_FKGS(15, 0.3)

len base 0: 1657
done A
done B
done C
len base 1: 1657
done A
done B
done C
len base 2: 1658
done A
done B
done C
len base 3: 1659
done A
done B
done C
len base 4: 1658
done A
done B
done C
len base 5: 1658
done A
done B
done C
len base 6: 1658
done A
done B
done C
len base 7: 1658
done A
done B
done C
len base 8: 1658
done A
done B
done C
len base 9: 1657
done A
done B
done C
sampling_time: [0.6722195148468018, 0.6579995155334473, 0.6615524291992188, 0.6752035617828369, 0.6763589382171631, 0.6667613983154297, 0.664639949798584, 0.6669940948486328, 0.6630067825317383, 0.6583404541015625]
train_time: [61.343244791030884, 60.55197882652283, 60.48020625114441, 60.64477586746216, 61.051310300827026, 60.25638437271118, 61.486626386642456, 60.769346952438354, 61.19360685348511, 59.03197622299194]
test_time: [48.705485343933105, 48.16239547729492, 48.36232566833496, 48.175825119018555, 48.282427072525024, 48.16932725906372, 48.353965282440186, 48.25807595252991, 48.40153741836548, 47.84969305

In [15]:
scenario_FKGS(20, 0.1)

len base 0: 2209
done A
done B
done C
len base 1: 2210
done A
done B
done C
len base 2: 2214
done A
done B
done C
len base 3: 2213
done A
done B
done C
len base 4: 2210
done A
done B
done C
len base 5: 2209
done A
done B
done C
len base 6: 2209
done A
done B
done C
len base 7: 2211
done A
done B
done C
len base 8: 2210
done A
done B
done C
len base 9: 2214
done A
done B
done C
sampling_time: [0.9668443202972412, 0.9406445026397705, 0.9411306381225586, 0.9431595802307129, 0.9439027309417725, 0.9342555999755859, 0.9440164566040039, 0.9354462623596191, 0.9430255889892578, 0.9545712471008301]
train_time: [124.97212886810303, 127.33342170715332, 125.33518314361572, 128.2138991355896, 125.82916736602783, 123.05017971992493, 129.9477698802948, 125.57767271995544, 122.9033932685852, 126.31520986557007]
test_time: [64.23890447616577, 66.37452697753906, 66.18941974639893, 66.3770227432251, 66.55470180511475, 65.9466187953949, 66.58449935913086, 66.07168245315552, 65.84462022781372, 66.1967759132

In [16]:
scenario_FKGS(20, 0.2)

len base 0: 2209
done A
done B
done C
len base 1: 2210
done A
done B
done C
len base 2: 2210
done A
done B
done C
len base 3: 2209
done A
done B
done C
len base 4: 2210
done A
done B
done C
len base 5: 2209
done A
done B
done C
len base 6: 2209
done A
done B
done C
len base 7: 2212
done A
done B
done C
len base 8: 2209
done A
done B
done C
len base 9: 2210
done A
done B
done C
sampling_time: [1.0275120735168457, 1.0292422771453857, 1.0095925331115723, 1.0362484455108643, 1.0339107513427734, 1.0032932758331299, 1.0299804210662842, 1.0338547229766846, 1.043100357055664, 1.0129494667053223]
train_time: [103.45014500617981, 102.89969182014465, 104.19828939437866, 103.85070371627808, 104.2224850654602, 104.00844025611877, 105.2383131980896, 104.64125394821167, 105.40135884284973, 104.81968188285828]
test_time: [64.2079827785492, 63.82404112815857, 64.06300139427185, 63.98793816566467, 63.693068504333496, 63.724507570266724, 64.11303043365479, 63.76850867271423, 64.17220067977905, 63.7848210

In [17]:
scenario_FKGS(20, 0.3)

len base 0: 2210
done A
done B
done C
len base 1: 2210
done A
done B
done C
len base 2: 2209
done A
done B
done C
len base 3: 2210
done A
done B
done C
len base 4: 2210
done A
done B
done C
len base 5: 2209
done A
done B
done C
len base 6: 2210
done A
done B
done C
len base 7: 2209
done A
done B
done C
len base 8: 2210
done A
done B
done C
len base 9: 2209
done A
done B
done C
sampling_time: [1.2115437984466553, 1.2004568576812744, 1.1945674419403076, 1.1792645454406738, 1.2273228168487549, 1.1897640228271484, 1.2074413299560547, 1.2173070907592773, 1.2097749710083008, 1.193692922592163]
train_time: [109.97708034515381, 109.65292501449585, 110.6867425441742, 108.5369656085968, 110.26313424110413, 109.97212386131287, 111.02684283256531, 112.58806562423706, 109.65854573249817, 111.41757297515869]
test_time: [64.58567690849304, 64.91885018348694, 64.82759189605713, 64.66362285614014, 65.04995608329773, 64.67059683799744, 64.95199871063232, 65.54426693916321, 64.88896250724792, 65.31064319

In [18]:
scenario_random(10)

len base 0: 1104
done A
done B
done C
len base 1: 1104
done A
done B
done C
len base 2: 1104
done A
done B
done C
len base 3: 1104
done A
done B
done C
len base 4: 1104
done A
done B
done C
len base 5: 1104
done A
done B
done C
len base 6: 1104
done A
done B
done C
len base 7: 1104
done A
done B
done C
len base 8: 1104
done A
done B
done C
len base 9: 1104
done A
done B
done C
sampling_time: [0.0010020732879638672, 0.0010008811950683594, 0.0010001659393310547, 0.0, 0.0010738372802734375, 0.0, 0.0010006427764892578, 0.0010001659393310547, 0.0010006427764892578, 0.00099945068359375]
train_time: [31.63884997367859, 32.6949622631073, 31.628087759017944, 31.65804362297058, 31.43217444419861, 32.54971885681152, 31.454663038253784, 31.73047113418579, 31.996506690979004, 31.667548894882202]
test_time: [33.06862258911133, 33.39684247970581, 33.10987973213196, 33.1240770816803, 32.93710398674011, 33.232237339019775, 32.91158437728882, 33.32978367805481, 33.01451539993286, 33.080440521240234]
acc

In [19]:
scenario_random(15)

len base 0: 1656
done A
done B
done C
len base 1: 1656
done A
done B
done C
len base 2: 1656
done A
done B
done C
len base 3: 1656
done A
done B
done C
len base 4: 1656
done A
done B
done C
len base 5: 1656
done A
done B
done C
len base 6: 1656
done A
done B
done C
len base 7: 1656
done A
done B
done C
len base 8: 1656
done A
done B
done C
len base 9: 1656
done A
done B
done C
sampling_time: [0.0010008811950683594, 0.0010001659393310547, 0.0010030269622802734, 0.0009961128234863281, 0.0010006427764892578, 0.0009970664978027344, 0.0010008811950683594, 0.001001119613647461, 0.0010001659393310547, 0.0009932518005371094]
train_time: [70.87620186805725, 70.69402766227722, 70.51283025741577, 71.05258512496948, 70.64293432235718, 70.86356019973755, 71.23171043395996, 70.6419575214386, 69.44673347473145, 70.4796576499939]
test_time: [49.960307121276855, 49.70488095283508, 49.62079453468323, 49.60690903663635, 49.647048473358154, 49.661951541900635, 49.73228192329407, 49.80994629859924, 49.4945

In [20]:
scenario_random(20)

len base 0: 2208
done A
done B
done C
len base 1: 2208
done A
done B
done C
len base 2: 2208
done A
done B
done C
len base 3: 2208
done A
done B
done C
len base 4: 2208
done A
done B
done C
len base 5: 2208
done A
done B
done C
len base 6: 2208
done A
done B
done C
len base 7: 2208
done A
done B
done C
len base 8: 2208
done A
done B
done C
len base 9: 2208
done A
done B
done C
sampling_time: [0.0009996891021728516, 0.0010004043579101562, 0.0010013580322265625, 0.0020003318786621094, 0.0020020008087158203, 0.0010001659393310547, 0.0009992122650146484, 0.001001119613647461, 0.0009996891021728516, 0.0009930133819580078]
train_time: [123.92884635925293, 127.00571751594543, 125.58078479766846, 125.19375944137573, 123.44265913963318, 123.93126583099365, 127.61754822731018, 122.145822763443, 128.2824785709381, 123.36105012893677]
test_time: [64.34854435920715, 66.65181064605713, 66.52706623077393, 66.32018566131592, 66.06056356430054, 66.48672223091125, 66.55920839309692, 65.99729061126709, 6